In [25]:
from enhanced_hybrid_hhl import (RandomQLSP,
                                QPE_preprocessing,
                                QuantumLinearSystemSolver, 
                                ideal_preprocessing,
                                ExampleQLSP,
                                HHL,
                                HybridInversion,
                                HHL_Result)

from qiskit import transpile, QuantumCircuit
import numpy as np
from qiskit.circuit.library import HamiltonianGate

In [80]:
from qiskit_ionq.ionq_provider import IonQProvider
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
ionq_provider = IonQProvider(token='W8fkNnlIqaWDP83QCPTnP5HjoELVXMbP')
ionq_simulator = ionq_provider.get_backend('ionq_simulator')

def get_ionq_results_preprocessing(circ):
    transp = transpile(circ, ionq_simulator)
    result = ionq_simulator.run(transp).result()
    return result

def get_ionq_results_hhl():
    def SwapTest(num_state_qubits):
        num_qubits = 2*num_state_qubits+1
        st_circ = QuantumCircuit(num_qubits)
        st_circ.h(-1)
        for i in range(num_state_qubits):
            st_circ.cswap(-1,i,num_state_qubits+i)
        st_circ.h(-1)
        return st_circ

    def st_post_processing(result = None, counts_01=None, counts_11=None):
        if not result==None:
            if '0 1' in result.keys():
                counts_01 = result['0 1']
            else:
                counts_01 = 0
                
            if '1 1' in result.keys():
                counts_11 = result['1 1']
            else:
                if counts_01 == 0:
                    counts_11 = 1
                else:
                    counts_11 = 0
        
        prob_0 = counts_01/(counts_01+counts_11)
        if prob_0 == 0:
            return 0
        return np.sqrt(2*prob_0 - 1)

    def get_result(hhl_circ, problem) -> HHL_Result:
        num_b_qubits = int(np.log2(len(problem.b_vector)))

        st = SwapTest(num_b_qubits)
        q_reg = QuantumRegister(st.num_qubits-num_b_qubits)
        c_reg = ClassicalRegister(1)

        hhl_circ.add_register(q_reg)
        hhl_circ.add_register(c_reg)

        statevector = QuantumLinearSystemSolver(problem=problem).ideal_x_statevector

        hhl_circ.prepare_state(statevector, q_reg[:-1])
        hhl_circ.append(st, list(range(-st.num_qubits,0)))
        hhl_circ.measure(0,0)
        hhl_circ.measure(-1,c_reg[0])

        circuit = transpile(hhl_circ, ionq_simulator)
        
        circuit_result = ionq_simulator.run(circuit).result().get_counts()
        result_processed = st_post_processing(result = circuit_result)

        result = HHL_Result()
        result.circuit_results = circuit_result
        result.results_processed = result_processed
        return result
    return get_result

In [81]:
problem = ExampleQLSP(0.33)
maximum_eigenvalue = max(ideal_preprocessing(problem)[0], key=abs)
clock = 3

In [82]:
preprocessing = QPE_preprocessing(clock, 
                                  maximum_eigenvalue, 
                                  get_ionq_results_preprocessing).estimate

inversion_circuit = HybridInversion

get_result = get_ionq_results_hhl()

ionq_hhl = HHL(get_result = get_result,
               pre_processing=preprocessing,
               eigenvalue_inversion=inversion_circuit)


In [83]:
result = ionq_hhl.estimate(problem=problem,
                  num_clock_qubits=clock,
                  max_eigenvalue=maximum_eigenvalue,
                  quantum_conditional_logic=False)

In [84]:
print(result)

{   'circuit_results': {'0 0': 66, '0 1': 877, '1 0': 35, '1 1': 46},
    'eigenbasis_projection_list': [0.16796875, 0.2158203125, 0.53125],
    'eigenvalue_list': [   0.4466666666666666,
                           -0.8933333333333332,
                           -0.4466666666666666],
    'ideal_x_statevector': None,
    'post_processing': None,
    'results_processed': 0.9488545869023296,
    'rotation_angle_list': None,
    'shots': None}


In [60]:
test_result.get_counts()

{'0 0': 946, '1 0': 78}